# new code

In [1]:
# importing libraries
import cv2
import numpy as np
import time

# capturing or reading video
# cap = cv2.VideoCapture('cars.mp4')
cap = cv2.VideoCapture('/Users/analeegraig/Documents/Computer Vision/final_project_data/cars.mp4')

count_line_position = 600

# minimum contour width
min_contour_width=30 

# minimum contour height
min_contour_height=30 

# initial background subtractor
algo = cv2.createBackgroundSubtractorMOG2()

# define centroid function
def get_centroid(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx,cy

detect = []
offset = 6
counter = 0

# adding the following variables for speed calculation
fps_start_time = time.time()
fps = 0
total_frames = 0
speed_kph = 0
start_point = (0,0)

# calculate speed
def calculate_speed(start_point, end_point, fps):
    distance_pixels = abs(end_point[0] - start_point[0])
    distance_meters = distance_pixels / pixels_per_meter 
    speed_mps = distance_meters / fps
    speed_kph = speed_mps * 3.6
    return speed_kph * 0.621371192

pixels_per_meter = 10

while True:
    ret, frame1 = cap.read()
    grey = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey, (3, 3), 5)
    
    # applying on each frame
    img_sub = algo.apply(blur)
    dilat = cv2.dilate(img_sub, np.ones((5, 5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dilatada = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel)
    dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_CLOSE, kernel)
    contourSahpe, h = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.line(frame1, (5, count_line_position), (1900, count_line_position), (255,127,0), 1)
    
    for(i,c) in enumerate(contourSahpe):
        (x,y,w,h) = cv2.boundingRect(c)
        contour_valid = (w >= min_contour_width) and (h >= min_contour_height)
        if not contour_valid:
            continue
        
        cv2.rectangle(frame1, (x,y), (x+w,y+h), (0,255,0), 2)
        #cv2.putText(frame1, "Vehicles" + str(counter), (x, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 244, 0), 2)
        
        centroid = get_centroid(x, y, w, h)
        detect.append(centroid)
        cv2.circle(frame1,centroid, 4, (0, 0, 255), -1)
        
        #cx,cy= get_centroid(x, y, w, h)
        for (x, y) in detect:
            if y < (count_line_position + offset) and y > (count_line_position - offset):
                counter += 1
                cv2.line(frame1, (5, count_line_position), (700, count_line_position), (0, 127, 255), 1)
                detect.remove((x, y))

                # Calculate speed
                end_point = (x, y)
                speed_kph = calculate_speed(start_point, end_point, fps)

        cv2.putText(frame1, f"V{counter}: {speed_kph:.2f} mph", (x-30, y-30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

                
    # Update fps and total frames
    total_frames += 1
    fps_end_time = time.time()
    fps = total_frames / (fps_end_time - fps_start_time)

    cv2.imshow('original', frame1)

    # define keyboard break
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    
    
cv2.destroyAllWindows()
cap.release()
